In [37]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor 
from sklearn.linear_model import Lasso,Ridge, LinearRegression
from xgboost import XGBRegressor
df=pd.read_csv('../input/cardata/car data.csv')

In [2]:
df.isnull().sum()

In [3]:
df.head()

In [4]:
df.shape

In [5]:
df.Fuel_Type.value_counts()

In [6]:
print(df['Fuel_Type'].unique())

In [7]:
df= df.drop('Car_Name',axis=1)

In [8]:
df['current_year'] = 2022
df['years_old'] = df['current_year'] - df['Year']
df.head()

In [9]:
df.drop(['Year'], axis=1,inplace=True)

In [10]:
df=pd.get_dummies(df,drop_first=True)

In [11]:
df.head()

In [12]:
df.drop(['current_year'], axis=1,inplace=True)

In [13]:
import seaborn as sns

In [14]:
#to find the correlation
df.corr()

In [15]:
#sns.pairplot(df)
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
cor = df.corr()
top_corr= cor.index
plt.figure(figsize=(15,10))
g = sns.heatmap(df[top_corr].corr(),annot=True)

In [17]:
#dividing X and y
X= df.iloc[:,1:]
y=df.iloc[:,0]

In [18]:
#feature importance
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [19]:
print(model.feature_importances_)

In [20]:
#plotting the features in visualized way
imp_features=pd.Series(model.feature_importances_,index=X.columns)
imp_features.nlargest(7).plot(kind='barh')
plt.show()

In [21]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2)

In [22]:
#tuning the randomforestregressor
n_estimators = [int(x) for x in np.linspace(start=80, stop=400, num=40)]

max_features = ['auto','sqrt']

max_depth = [int(x) for x in np.linspace(5,28,num=10)]
min_samples_leaf = [1, 2, 3, 4]

min_samples_split = [2,5,10,15,100]

min_samples_leaf = [1,2,5,10]
bootstrap = [True, False]


In [23]:
from sklearn.model_selection import RandomizedSearchCV

In [24]:
RF_grid = {'n_estimators': n_estimators,
          'max_features':max_features,
            'max_depth':max_depth,
           'min_samples_split':min_samples_split,
           'min_samples_leaf':min_samples_leaf,
           'bootstrap': bootstrap,
           'min_samples_split':min_samples_split
          }


print(RF_grid)

In [25]:
forest= RandomForestRegressor()

In [26]:
rf_grid = RandomizedSearchCV(forest, param_distributions=RF_grid,scoring='neg_mean_squared_error',
                             cv=5,random_state=101,n_jobs=1) #,verbose=2

In [27]:
#verbose is ussed to show the details below, i dont want 
rf_grid.fit(X_train,y_train)

In [28]:
prediction= rf_grid.predict(X_test)

In [29]:
prediction

In [30]:
sns.distplot(y_test-prediction)

In [31]:
#linearly available and formed gaussian dist means predicions are good
plt.scatter(y_test,prediction)

In [32]:
from sklearn.metrics import mean_squared_error, r2_score
print(mean_squared_error(y_test,prediction))

In [33]:
print(r2_score(y_test,prediction))

#  **Using Gradient, Ridge, Lasso**

In [39]:
models = {'Lasso':Lasso(),'Linear':LinearRegression(),'Ridge':Ridge(),'xgboost':XGBRegressor()}

In [40]:
for model_name, model in models.items():
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print(f'Mean Squared Error for {model_name}')
    print(mean_squared_error(y_test,y_pred))
    print(f'r2 Score for {model_name}')
    print(r2_score(y_test,y_pred)*100)
    print('----------------------')

In [ ]:
import pickle

filename = 'random_forest_carprice.pkl'


file = open ('random_forest_carprice','wb')

pickle.dump(rf_grid,file)

filename = 'carprice_forest.pkl'
pickle.dump(rf_grid,open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open('carprice_forest.pkl','rb'))